In [0]:
!pip install unidecode
!pip install --upgrade pip
import requests,json,pandas as pd,datetime as dt,time,inspect,os,logging,boto3,urllib,zipfile as zf
from pyspark.sql.functions import *
from botocore.client import Config
from botocore.exceptions import ClientError
from math import trunc

     |████████████████████████████████| 235 kB 4.5 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 2.0 MB 5.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
      Successfully uninstalled pip-21.0.1


In [0]:
#LOCATION = dbutils.widgets.text('LOCATION','')
LOCATION = dbutils.widgets.get("LOCATION")
#FILE_PATH = dbutils.widgets.text('FILE_PATH','')
PATH = dbutils.widgets.get("FILE_PATH")

In [0]:
def get_properties(location,start_year=2020,start_month=1,number_months=12,currency='USD',min_bed=1,max_bed=20):
  url = "https://airdna1.p.rapidapi.com/properties"
  querystring ={}
  querystring["location"] = location
  querystring["start_year"] = start_year
  querystring["start_month"] = start_month
  querystring["number_of_months"] = number_months
  querystring["currency"] = currency 
  querystring["bedrooms_min"] = min_bed
  querystring["bedrooms_max"] = max_bed
#   querystring["accommodates_min"] = accommodates_min
#   querystring["accommodates_max"] = accommodates_max
  
  headers = {
      "X-RapidAPI-Key": "YOUR API KEY",
      "X-RapidAPI-Host": "airdna1.p.rapidapi.com"
  }
  print('Parameters used: ',querystring,'\n')
  response = requests.get(url, headers=headers, params=querystring)
  response = response.json()
  return response


def suggested_property(location):
  url = "https://airdna1.p.rapidapi.com/suggestion"
  querystring = {}
  querystring['location'] = location

  headers = {
      "X-RapidAPI-Key": "YOUR API KEY",
      "X-RapidAPI-Host": "airdna1.p.rapidapi.com"
  }
  response = requests.get(url, headers=headers, params=querystring)
  response = response.json()
  return response


def get_top_accounts(location):
  url = "https://airdna1.p.rapidapi.com/top_managers"

  querystring = {"location":""}
  querystring['location'] = location

  headers = {
      "X-RapidAPI-Key": "YOUR API KEY",
      "X-RapidAPI-Host": "airdna1.p.rapidapi.com"
  }

  response = requests.get(url, headers=headers, params=querystring).json()
  return response

def overview(location,currency='USD'):
  url = "https://airdna1.p.rapidapi.com/overview"

  querystring = {}
  querystring["location"] = location
  querystring["currency"] = currency

  headers = {
      "X-RapidAPI-Key": "YOUR API KEY",
      "X-RapidAPI-Host": "airdna1.p.rapidapi.com"
  }
  response = requests.get(url, headers=headers, params=querystring).json()
  return response


def active_listings(location,start_year=2022,start_month=1,number_of_months=12):
  url = "https://airdna1.p.rapidapi.com/activelistings"
  querystring={}
  querystring["location"] = location
  querystring["start_year"] = start_year
  querystring["start_month"] = start_month
  querystring["number_of_months"] = number_of_months
  headers = {
      "X-RapidAPI-Key": "YOUR API KEY",
      "X-RapidAPI-Host": "airdna1.p.rapidapi.com"
  }

  response = requests.get(url, headers=headers, params=querystring).json()
  return response

def rental_settings(location,currency='USD'):
  url = "https://airdna1.p.rapidapi.com/rentalsettings"

  querystring = {}
  querystring['location']=location
  querystring['currency']=currency
  
  headers = {
      "X-RapidAPI-Key": "YOUR API KEY",
      "X-RapidAPI-Host": "airdna1.p.rapidapi.com"
  }

  response = requests.get(url, headers=headers, params=querystring).json()
  return response

def get_ratings(location,currency='USD'):

  url = "https://airdna1.p.rapidapi.com/rentalratings"

  querystring = {}
  querystring['location']=location
  querystring['currency']=currency

  headers = {
      "X-RapidAPI-Key": "YOUR API KEY",
      "X-RapidAPI-Host": "airdna1.p.rapidapi.com"
  }

  response = requests.get(url, headers=headers, params=querystring).json()
  return response

def get_amenities(location,currency='USD'):
  url = "https://airdna1.p.rapidapi.com/rentalamenities"

  querystring = {}
  querystring['location']=location
  querystring['currency']=currency

  headers = {
      "X-RapidAPI-Key": "YOUR API KEY",
      "X-RapidAPI-Host": "airdna1.p.rapidapi.com"
  }

  response = requests.get(url, headers=headers, params=querystring).json()
  return response


In [0]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def process_listings(location):
  listings=get_properties(location)
  city = listings['area_info']['geom']['code']['city'].capitalize()
  country = listings['area_info']['geom']['code']['country'].upper()
  state = listings['area_info']['geom']['code']['state'].capitalize()
  num_listings = listings['listings_returned']
  df = pd.json_normalize(listings['properties'])
  df['country'] = country
  df['city'] = city
  df['state'] = state
  #df['number_of_listings'] = num_listings
  print('Number of listings in ', location,': ', num_listings)
  return df

def process_top_accounts(location):
  top_accounts=get_top_accounts(LOCATION)
  df = pd.DataFrame(top_accounts['property_managers'][:])
  df['city'] = LOCATION
  return df

def process_rental_settings(location):
  rental_rules=rental_settings(LOCATION)
  city = rental_rules['area_info']['geom']['code']['city'].capitalize()
  country = rental_rules['area_info']['geom']['code']['country'].upper()
  state = rental_rules['area_info']['geom']['code']['state'].capitalize()
  cancel_policy = rental_rules['rental_settings']['cancellation']['percentages']
  cancel_policy_df = pd.DataFrame(cancel_policy,index=[''])
  cancel_policy_df['country'] = country
  cancel_policy_df['city'] = city
  cancel_policy_df['state'] = state
  min_stay = rental_rules['rental_settings']['min_stay']['bins']
  min_stay_df = pd.DataFrame(min_stay,index=[''])
  min_stay_df['country'] = country
  min_stay_df['city'] = city
  min_stay_df['state'] = state
  return cancel_policy_df, min_stay_df

def process_amenities(location):
  area_amenities=get_amenities(LOCATION)
  city = area_amenities['area_info']['geom']['code']['city'].capitalize()
  country = area_amenities['area_info']['geom']['code']['country'].upper()
  state = area_amenities['area_info']['geom']['code']['state'].capitalize()
  amenities = area_amenities['rental_amenities']['percentages']
  amenities_df = pd.DataFrame(amenities,index=[''])
  amenities_df['country'] = country
  amenities_df['city'] = city
  amenities_df['state'] = state
  return amenities_df

def process_ratings(location):
  listing_ratings=get_ratings(LOCATION)
  city = listing_ratings['area_info']['geom']['code']['city'].capitalize()
  country = listing_ratings['area_info']['geom']['code']['country'].upper()
  state = listing_ratings['area_info']['geom']['code']['state'].capitalize()
  ratings_by_type = listing_ratings['rental_ratings']['rating_averages']
  ratings_by_type_df = pd.DataFrame(ratings_by_type,index=[''])
  ratings_by_type_df['country'] = country
  ratings_by_type_df['city'] = city
  ratings_by_type_df['state'] = state
  ratings_counts = listing_ratings['rental_ratings']['rating_count']['bins']
  ratings_counts_df = pd.DataFrame(ratings_counts,index=[''])
  ratings_counts_df['country'] = country
  ratings_counts_df['city'] = city
  ratings_counts_df['state'] = state
  return ratings_counts_df,ratings_by_type_df

def process_listings_overview(location):
  LOCATION=location
  listings_overview=overview(LOCATION)

  months = []
  years = []
  room_types = []
  median_adr = []
  median_occ = []
  median_revenue = []
  city = listings_overview['area_info']['geom']['code']['city'].capitalize()
  country = listings_overview['area_info']['geom']['code']['country'].upper()
  state = listings_overview['area_info']['geom']['code']['state'].capitalize()

  for i in listings_overview['data']['main']['calendar_months']:
    months.append(i['month'])
    years.append(i['year'])
    room_type = str(i['room_type'].keys())
    room_types.append(room_type[12:-3])
    median_adr.append(i['room_type']['entire_place']['adr']['50th_percentile'])
    median_occ.append(i['room_type']['entire_place']['occ']['50th_percentile'])
    median_revenue.append(i['room_type']['entire_place']['revenue']['50th_percentile'])

  data={'month':months,'year':years,'room_type':room_types,'median_adr':median_adr,'median_occ':median_occ,'median_revenue':median_revenue}
  listings_df = pd.DataFrame(data)
  listings_df['city'] = city
  listings_df['country'] = country
  listings_df['state'] = state
  ltm_df = pd.DataFrame(listings_overview['data']['main']['ltm_totals'],index = [''])

  listings_df['ltm_adr'] = float(ltm_df['ltm_adr'])
  listings_df['ltm_occ'] = float(ltm_df['ltm_occ'])
  listings_df['ltm_revenue'] = float(ltm_df['ltm_revenue'])
  listings_df['currency'] = listings_overview['request_info']['currency'].upper()
  listings_df['city_id'] = int(listings_overview['request_info']['city_id'])

  average_df = pd.DataFrame(listings_overview['data']['rental_counts']['average'],index=[''])
  average_df = average_df.rename(columns={"accommodates": "average_accommodate_num", "bedrooms": "average_bedrooms"})
  average_df['city'] = city
  average_df['country'] = country
  average_df['state'] = state

  market_grade_df = pd.DataFrame(listings_overview['data']['market_grade'],index=[''])
  rental_activity_df = pd.DataFrame(listings_overview['data']['rental_activity'])
  rental_counts_df = pd.DataFrame(listings_overview['data']['rental_counts']['counts'])
  rental_counts_df['city'] = city
  rental_counts_df['country'] = country
  rental_counts_df['state'] = state
  rental_activity_df['city'] = city
  rental_activity_df['country'] = country
  rental_activity_df['state'] = state

  rental_growth = []
  date = []
  for i in listings_overview['data']['rental_growth']:
    month = str(i.keys())
    month = month[12:-3]
    growth = str(i.values())
    growth = growth[13:-2]
    date.append(month)
    rental_growth.append(growth)

  rental_data={'date':date,'growth':rental_growth}
  rental_growth_df = pd.DataFrame(rental_data)
  rental_growth_df['city'] = city
  rental_growth_df['country'] = country
  rental_growth_df['state'] = state

  return  listings_df,average_df,market_grade_df,rental_activity_df,rental_counts_df,rental_growth_df

def process_active_listings(location):
  LOCATION = location
  active_listing=active_listings(LOCATION)
  months,years = [],[]
  df=pd.DataFrame()
  all_df =pd.DataFrame()
  private_room_df =pd.DataFrame()
  entire_place_df =pd.DataFrame()
  #shared_room_df = pd.DataFrame()
  for i in active_listing['data']['active_listings']['calendar_months']:
    months.append(i['month'])
    years.append(i['year'])
    all_rooms = i['room_type']['all']['bedrooms']
    entire_place = i['room_type']['entire_place']['bedrooms']
    private_room = i['room_type']['private_room']['bedrooms']
    #shared_room = i['room_type']['shared_room']['bedrooms']
    all_rooms_df = pd.DataFrame(all_rooms)
    all_df=all_df.append(all_rooms_df)
    entire_df = pd.DataFrame(entire_place)
    entire_place_df=entire_place_df.append(entire_df)
    private_rm_df = pd.DataFrame(private_room)
    private_room_df=private_room_df.append(private_rm_df)
    #shared_rm_df = pd.DataFrame(shared_room)
    #shared_room_df=shared_room_df.append(shared_rm_df)
    
  entire_place_df['month'] = months
  #shared_room_df['months'] = months
  private_room_df['months'] = months
  all_df['months'] = months
  entire_place_df['year'] = years
  #shared_room_df['year'] = years
  private_room_df['year'] = years
  all_df['year'] = years

  return entire_place_df,private_room_df,all_df


def get_shared_room_details(location):
  LOCATION=location
  active_listing=active_listings(LOCATION)
  df=pd.DataFrame()
  if active_listing['area_info']['geom']['code']['country'].upper()=='MX':
    months,years = [],[]
    shared_room_df = pd.DataFrame()
    for i in active_listing['data']['active_listings']['calendar_months']:
      months.append(i['month'])
      years.append(i['year'])
      shared_room = i['room_type']['shared_room']['bedrooms']
      shared_rm_df = pd.DataFrame(shared_room)
      shared_room_df=shared_room_df.append(shared_rm_df)


    shared_room_df['months'] = months
    shared_room_df['year'] = years
    return shared_room_df
  else:
    return df

def to_csv(df,name):
  now = dt.datetime.now()
  now = str(now)
  now = now[:-16]
  file_name = LOCATION+'_'+name+'_'+now+'.csv'
  df.to_csv(file_name)
  print('Completed! File name is: ',file_name)

def export_all_dataframes(path='/airbnb_data'):
  path = PATH
  df_names = ['listings','top_accounts_df','cancel_policy_df','min_stay_df','amenities_df','ratings_counts','ratings_by_type','listings_df','average_df','market_grade_df','rental_activity_df','rental_counts_df','rental_growth_df','entire_place_df','private_room_df','all_rooms_df','shared_room_df']
  dfs = [listings,top_accounts_df,cancel_policy_df,min_stay_df,amenities_df,ratings_counts,ratings_by_type,listings_df,average_df,market_grade_df,rental_activity_df,rental_counts_df,rental_growth_df,entire_place_df,private_room_df,all_rooms_df,shared_room_df]
  
  os.chdir(path)
  count = 0
  for i in range(0, len(dfs)):
    to_csv(dfs[i],df_names[i])
    count += 1 
  
  print('\n',color.BOLD, count,'out of' ,len(dfs),' files created for: ',LOCATION,color.END)
  print(color.BOLD,'File Landing Path: ',path,color.END)
  
def upload_to_s3(path='/airbnb_data'):
  path = os.chdir(PATH)
  path = os.getcwd()

  files = []
  for root, dirs, files in os.walk(path):
      for file in files:
        i=0
      files.append(file)


  ACCESS_KEY = 'YOUR ACCESS KEY'
  SECRET_KEY = 'YOUR SECRET KEY'
  AWS_BUCKET_NAME = "airbnb-silo"

  s3_client = boto3.client('s3')
  s3 = boto3.client('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key=SECRET_KEY)

  print(color.BOLD,'Number of files: ',len(files),color.END)
  print('')
  for file in files:
    f_path = path+'/'+file
    s3_path = 'data/'
    s3_path += file
    print(color.BOLD,'FILE_NAME: ',color.END,file,color.BOLD)
    print(color.BOLD, 'SOURCE_FILE_PATH: ', color.END, f_path)
    print(color.BOLD, 'AWS_S3_DESTINATION_FILE_PATH: ', color.END, s3_path)
    print('')
    response = s3.upload_file(f_path, AWS_BUCKET_NAME, s3_path)

  print('ALL FILES UPLOADED!')

def add_month_name(df):
  
  new_df = pd.DataFrame()
  new_df['month_name'] = ''

  for i in range(len(df['month'])):

    month = df['month']

    if month[i] == 8:
      new_df.loc[i,'month_name'] = 'August'
      new_df.loc[i,'month'] = int(month[i])
    if month[i] == 9:
      new_df.loc[i,'month_name'] = 'September'
      new_df.loc[i,'month'] = int(month[i])
    if month[i] == 10:
      new_df.loc[i,'month_name'] = 'October'
      new_df.loc[i,'month'] = int(month[i])
    if month[i] == 11:
      new_df.loc[i,'month_name'] = 'November'
      new_df.loc[i,'month'] = int(month[i])
    if month[i] == 12:
      new_df.loc[i,'month_name'] = 'December'
      new_df.loc[i,'month'] = int(month[i])
    if month[i] == 1:
      new_df.loc[i,'month_name'] = 'January'
      new_df.loc[i,'month'] = int(month[i])
    if month[i] == 2:
      new_df.loc[i,'month_name'] = 'February'
      new_df.loc[i,'month'] = int(month[i])
    if month[i] == 3:
      new_df.loc[i,'month_name'] = 'March'
      new_df.loc[i,'month'] = int(month[i])
    if month[i] == 4:
      new_df.loc[i,'month_name'] = 'April'
      new_df.loc[i,'month'] = int(month[i])
    if month[i] == 5:
      new_df.loc[i,'month_name'] = 'May'
      new_df.loc[i,'month'] = int(month[i])
    if month[i] == 6:
      new_df.loc[i,'month_name'] = 'June'
      new_df.loc[i,'month'] = int(month[i])
    if month[i] == 7:
      new_df.loc[i,'month_name'] = 'July'
      new_df.loc[i,'month'] = int(month[i])

  final = pd.merge(new_df,df, on='month',how = 'inner')
  final_df  = final.fillna(0)
  return final_df


In [0]:
listings = process_listings(LOCATION)
time.sleep(1.5)
top_accounts_df = process_top_accounts(LOCATION)
time.sleep(2)
cancel_policy_df,min_stay_df = process_rental_settings(LOCATION)
time.sleep(1.5)
amenities_df = process_amenities(LOCATION)
time.sleep(1.5)
ratings_counts,ratings_by_type=process_ratings(LOCATION)
time.sleep(1.5)
listings_df,average_df,market_grade_df,rental_activity_df,rental_counts_df,rental_growth_df = process_listings_overview(LOCATION)
time.sleep(1.5)
entire_place_df,private_room_df,all_rooms_df=process_active_listings(LOCATION)
time.sleep(1.5)
shared_room_df = get_shared_room_details(LOCATION)

Parameters used:  {'location': 'Boston', 'start_year': 2020, 'start_month': 1, 'number_of_months': 12, 'currency': 'USD', 'bedrooms_min': 1, 'bedrooms_max': 20} 

Number of listings in  Boston :  1390


In [0]:
listings = listings.fillna(0)
new_title = []
for index,listing in listings.iterrows():
  new_string = listing['title'].encode('ascii',errors='ignore')
  new_string = str(new_string)
  new_string = new_string[2:-1]
  new_title.append(new_string)

listings = listings.drop(columns=['title'])
listings['title'] = new_title

listings

,accommodates,adr,airbnb_property_id,bathrooms,bedrooms,days_available,homeaway_property_id,id,img_cover,latitude,...,rating,revenue,reviews,room_type,platforms.airbnb_property_id,platforms.homeaway_property_id,country,city,state,title
0,12,1498.69470,1043280144,3.0,3,364,2329145,1043280144,https://media.vrbo.com/lodging/69000000/681300...,42.351042,...,0.0,permission_denied,0,Entire home/apt,0,2329145,US,Boston,Massachusetts,Just Steps Away from America''s Oldest Public ...
1,8,1020.78845,45531759,2.5,4,302,0,183143203,https://a0.muscache.com/im/pictures/645f1aa6-b...,42.341630,...,10.0,permission_denied,16,Entire home/apt,45531759,0,US,Boston,Massachusetts,Luxury Brownstone Penthouse Duplex
2,12,1438.38040,1043202117,3.0,3,362,2329142,1043202117,https://media.vrbo.com/lodging/69000000/681300...,42.351042,...,0.0,permission_denied,0,Entire home/apt,0,2329142,US,Boston,Massachusetts,"Located in Midtown! 3 Modish Suites, Onsite Di..."
3,12,1412.50540,1042193029,3.0,3,359,2329143,1042193029,https://media.vrbo.com/lodging/69000000/681300...,42.351042,...,0.0,permission_denied,0,Entire home/apt,0,2329143,US,Boston,Massachusetts,"Near Historic Landmarks, 3 Modern Units, Pet-f..."
4,8,809.08170,48093229,3.5,4,362,0,409257512,https://a0.muscache.com/im/pictures/c30a2856-b...,42.349640,...,9.2,permission_denied,89,Entire home/apt,48093229,0,US,Boston,Massachusetts,Bi-Level Fenway Home w/ Roofdeck & Laundry in ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385,3,102.00000,45154774,1.0,1,31,0,176990205,https://a0.muscache.com/im/pictures/a4re/floor...,42.371010,...,0.0,permission_denied,0,Entire home/apt,45154774,0,US,Boston,Massachusetts,A place of your own | 1BR in Boston
1386,1,20.00000,40364138,1.5,1,60,0,59896977,https://a0.muscache.com/im/pictures/46789bc6-c...,42.336310,...,0.0,permission_denied,0,Shared room,40364138,0,US,Boston,Massachusetts,Shared room 5 mins from Northeastern University
1387,1,94.00000,53473211,1.5,1,19,0,593611150,https://a0.muscache.com/im/pictures/miso/Hosti...,42.378860,...,0.0,permission_denied,0,Private room,53473211,0,US,Boston,Massachusetts,Modern Room in East Boston
1388,1,35.00000,47272572,1.0,1,36,0,337091442,https://a0.muscache.com/im/pictures/4cc7b49d-b...,42.342190,...,0.0,permission_denied,0,Private room,47272572,0,US,Boston,Massachusetts,Long term stay Grad Students/Professionals/Art...


In [0]:
top_accounts_df

,avg_rating,listings_change,name,total_listings,total_reviews,city
0,4.256410,1.202020,Blueground,218,39,Boston
1,4.622596,0.000000,Maverick Suites,55,9028,Boston
2,4.292097,-0.140625,Thatch,55,930,Boston
3,4.451365,0.189189,Evonify,44,1245,Boston
4,4.627278,0.225806,"Healing Homes of Boston, LLC",38,2590,Boston


In [0]:
cancel_policy_df

,flexible,moderate,n/a,strict,super_strict,country,city,state
,0.1658,0.2035,0.1542,0.4653,0.0111,US,Boston,Massachusetts


In [0]:
min_stay_df

,1,2,3,30+,4-6,7-29,country,city,state
,659,321,125,341,42,316,US,Boston,Massachusetts


In [0]:
amenities_df

,aircon,cabletv,dryer,heating,hottub,internet,kitchen,parking,pool,washer,country,city,state
,0.894,0.2065,0.6285,0.8713,0.0207,0.9541,0.9081,0.1181,0.1065,0.7829,US,Boston,Massachusetts


In [0]:
ratings_counts

,4.0,4.1,4.199999999999999,4.299999999999999,4.399999999999999,4.499999999999998,4.599999999999998,4.6999999999999975,4.799999999999997,4.899999999999997,4.9999999999999964,country,city,state
,360,6,94,80,40,181,72,184,292,89,352,US,Boston,Massachusetts


In [0]:
ratings_by_type

,rating_accuracy,rating_checkin,rating_cleanliness,rating_communication,rating_location,rating_overall,rating_value,country,city,state
,4.6,4.72,4.6,4.68,4.84,4.25,4.43,US,Boston,Massachusetts


In [0]:
listings_df = listings_df.fillna(0)
listings_df

,month,year,room_type,median_adr,median_occ,median_revenue,city,country,state,ltm_adr,ltm_occ,ltm_revenue,currency,city_id
0,8,2021,entire_place,232.876185,0.826087,4368.500000,Boston,US,Massachusetts,248.0,0.741935,4048.0,USD,66752
1,9,2021,entire_place,244.000000,0.766667,4735.166748,Boston,US,Massachusetts,248.0,0.741935,4048.0,USD,66752
2,10,2021,entire_place,275.888867,0.774194,4835.000000,Boston,US,Massachusetts,248.0,0.741935,4048.0,USD,66752
3,11,2021,entire_place,219.205592,0.666667,3384.000000,Boston,US,Massachusetts,248.0,0.741935,4048.0,USD,66752
4,12,2021,entire_place,181.787717,0.619869,1989.000000,Boston,US,Massachusetts,248.0,0.741935,4048.0,USD,66752
5,1,2022,entire_place,203.557143,0.578784,2406.000000,Boston,US,Massachusetts,248.0,0.741935,4048.0,USD,66752
6,2,2022,entire_place,183.135417,0.616848,2679.125000,Boston,US,Massachusetts,248.0,0.741935,4048.0,USD,66752
7,3,2022,entire_place,222.069734,0.677419,3647.166626,Boston,US,Massachusetts,248.0,0.741935,4048.0,USD,66752
8,4,2022,entire_place,295.903846,0.689655,4418.571289,Boston,US,Massachusetts,248.0,0.741935,4048.0,USD,66752
9,5,2022,entire_place,285.411290,0.758621,5208.512695,Boston,US,Massachusetts,248.0,0.741935,4048.0,USD,66752


In [0]:
average_df

,average_accommodate_num,average_bedrooms,city,country,state
,3.7,1.3,Boston,US,Massachusetts


In [0]:
market_grade_df

,investability,regulation,rental_demand,revenue_growth,seasonality,total_score
,None,None,72.71,None,None,None


In [0]:
rental_activity_df

,available,booked,city,country,state
1-3,1481,1950,Boston,US,Massachusetts
10-12,439,171,Boston,US,Massachusetts
4-6,601,420,Boston,US,Massachusetts
7-9,251,231,Boston,US,Massachusetts


In [0]:
rental_counts_df = rental_counts_df.fillna(0)
rental_counts_df

,entire_place,hotel_room,private_room,shared_room,city,country,state
0,286.0,0.0,14.0,0.0,Boston,US,Massachusetts
1,568.0,28.0,209.0,7.0,Boston,US,Massachusetts
2,339.0,0.0,5.0,0.0,Boston,US,Massachusetts
3,97.0,0.0,0.0,0.0,Boston,US,Massachusetts
4,38.0,0.0,0.0,0.0,Boston,US,Massachusetts
5,5.0,0.0,0.0,0.0,Boston,US,Massachusetts
all,1333.0,28.0,228.0,7.0,Boston,US,Massachusetts


In [0]:
rental_growth_df

,date,growth,city,country,state
0,2014-04-01,401,Boston,US,Massachusetts
1,2014-07-01,836,Boston,US,Massachusetts
2,2014-10-01,1260,Boston,US,Massachusetts
3,2015-01-01,1320,Boston,US,Massachusetts
4,2015-04-01,1755,Boston,US,Massachusetts
5,2015-07-01,1912,Boston,US,Massachusetts
6,2015-10-01,2011,Boston,US,Massachusetts
7,2016-01-01,2134,Boston,US,Massachusetts
8,2016-04-01,2533,Boston,US,Massachusetts
9,2016-07-01,2685,Boston,US,Massachusetts


In [0]:
entire_place_df = add_month_name(entire_place_df)
entire_place_df

,month_name,month,0,1,2,3,4,5,6,7,all,year
0,August,8.0,309,584,304,94,38,5,2.0,1,1337,2021
1,September,9.0,292,527,281,89,33,6,2.0,1,1231,2021
2,October,10.0,323,524,325,90,36,5,0.0,1,1304,2021
3,November,11.0,289,514,269,83,36,5,0.0,1,1197,2021
4,December,12.0,325,667,276,85,47,4,0.0,1,1405,2021
5,January,1.0,261,478,255,80,46,4,0.0,1,1125,2022
6,February,2.0,213,418,219,71,40,4,0.0,1,966,2022
7,March,3.0,266,500,275,77,39,4,0.0,1,1162,2022
8,April,4.0,270,604,304,84,38,3,0.0,1,1304,2022
9,May,5.0,322,658,356,107,39,3,0.0,1,1486,2022


In [0]:
if shared_room_df.empty == False:
  shared_room_df.rename(columns={'months':'month'},inplace = True)
  shared_room_df = add_month_name(shared_room_df)
  shared_room_df
  
elif shared_room_df.empty != False:
  print('No Data for this Category')

No Data for this Category


In [0]:
private_room_df.rename(columns={'months':'month'},inplace = True)
private_room_df = add_month_name(private_room_df)
private_room_df

,month_name,month,0,1,2,all,year
0,August,8.0,8,195,1,204,2021
1,September,9.0,10,157,1,168,2021
2,October,10.0,11,165,1,177,2021
3,November,11.0,13,160,1,174,2021
4,December,12.0,14,177,1,192,2021
5,January,1.0,16,176,4,196,2022
6,February,2.0,15,143,4,162,2022
7,March,3.0,14,190,4,208,2022
8,April,4.0,15,203,4,222,2022
9,May,5.0,15,215,3,233,2022


In [0]:
all_rooms_df.rename(columns={'months':'month'},inplace = True)
all_rooms_df = add_month_name(all_rooms_df)
all_rooms_df

,month_name,month,0,1,2,3,4,5,6,7,all,year
0,August,8.0,317,788,305,94,38,5,2.0,1,1550,2021
1,September,9.0,302,691,282,89,33,6,2.0,1,1406,2021
2,October,10.0,334,696,326,90,36,5,0.0,1,1488,2021
3,November,11.0,302,682,270,83,36,5,0.0,1,1379,2021
4,December,12.0,339,852,277,85,47,4,0.0,1,1605,2021
5,January,1.0,277,659,259,80,46,4,0.0,1,1326,2022
6,February,2.0,228,563,223,71,40,4,0.0,1,1130,2022
7,March,3.0,280,694,279,77,39,4,0.0,1,1374,2022
8,April,4.0,285,812,308,84,38,3,0.0,1,1531,2022
9,May,5.0,337,881,359,107,39,3,0.0,1,1727,2022


In [0]:
if os.path.exists(PATH) == True:
  export_all_dataframes(PATH)  
elif os.path.exists(PATH) == False:
  os.mkdir(PATH)
  export_all_dataframes(PATH)  


Completed! File name is:  Boston_listings_2022-08-17.csv
Completed! File name is:  Boston_top_accounts_df_2022-08-17.csv
Completed! File name is:  Boston_cancel_policy_df_2022-08-17.csv
Completed! File name is:  Boston_min_stay_df_2022-08-17.csv
Completed! File name is:  Boston_amenities_df_2022-08-17.csv
Completed! File name is:  Boston_ratings_counts_2022-08-17.csv
Completed! File name is:  Boston_ratings_by_type_2022-08-17.csv
Completed! File name is:  Boston_listings_df_2022-08-17.csv
Completed! File name is:  Boston_average_df_2022-08-17.csv
Completed! File name is:  Boston_market_grade_df_2022-08-17.csv
Completed! File name is:  Boston_rental_activity_df_2022-08-17.csv
Completed! File name is:  Boston_rental_counts_df_2022-08-17.csv
Completed! File name is:  Boston_rental_growth_df_2022-08-17.csv
Completed! File name is:  Boston_entire_place_df_2022-08-17.csv
Completed! File name is:  Boston_private_room_df_2022-08-17.csv
Completed! File name is:  Boston_all_rooms_df_2022-08-17.c

In [0]:
upload_to_s3(PATH)

 Number of files:  18 

 FILE_NAME:   Boston_ratings_counts_2022-08-17.csv 
 SOURCE_FILE_PATH:   /airbnb_data/Boston_ratings_counts_2022-08-17.csv
 AWS_S3_DESTINATION_FILE_PATH:   data/Boston_ratings_counts_2022-08-17.csv

 FILE_NAME:   Boston_rental_activity_df_2022-08-17.csv 
 SOURCE_FILE_PATH:   /airbnb_data/Boston_rental_activity_df_2022-08-17.csv
 AWS_S3_DESTINATION_FILE_PATH:   data/Boston_rental_activity_df_2022-08-17.csv

 FILE_NAME:   Boston_min_stay_df_2022-08-17.csv 
 SOURCE_FILE_PATH:   /airbnb_data/Boston_min_stay_df_2022-08-17.csv
 AWS_S3_DESTINATION_FILE_PATH:   data/Boston_min_stay_df_2022-08-17.csv

 FILE_NAME:   Boston_rental_growth_df_2022-08-17.csv 
 SOURCE_FILE_PATH:   /airbnb_data/Boston_rental_growth_df_2022-08-17.csv
 AWS_S3_DESTINATION_FILE_PATH:   data/Boston_rental_growth_df_2022-08-17.csv

 FILE_NAME:   Boston_amenities_df_2022-08-17.csv 
 SOURCE_FILE_PATH:   /airbnb_data/Boston_amenities_df_2022-08-17.csv
 AWS_S3_DESTINATION_FILE_PATH:   data/Boston_ameniti